## 참고 링크
- 데이터 전처리 참고 : https://dacon.io/competitions/official/235747/codeshare/3054
- 데이터 다운로드 : https://www.bigkinds.or.kr/v2/news/index.do 
- 뉴스 감성지수 분류 모델 참고 : https://sieon-dev.tistory.com/15
- 기사 제목으로 긍정 부정 중립 분류 모델 만들기 참고 : https://somjang.tistory.com/entry/Keras%EA%B8%B0%EC%82%AC-%EC%A0%9C%EB%AA%A9%EC%9D%84-%EA%B0%80%EC%A7%80%EA%B3%A0-%EA%B8%8D%EC%A0%95-%EB%B6%80%EC%A0%95-%EC%A4%91%EB%A6%BD-%EB%B6%84%EB%A5%98%ED%95%98%EB%8A%94-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EC%96%B4%EB%B3%B4%EA%B8%B0
- 자연어 처리 참고 : https://wikidocs.net/book/2155

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings("ignore")
import hanja
from hanja import hangul
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import os

## 1. 데이터 로드

In [3]:
# 작업경로 확인
print(os.getcwd())

c:\Workspace\HK_Project\7_자연어처리


In [8]:
os.makedirs("c:/Workspace/HK_Project/7_자연어처리/data")

In [12]:
# 파일 경로 설정
filePath = "c:/Workspace/HK_Project/7_자연어처리/data/bigkinds_NewsData.xlsx"

In [ ]:
# 디렉토리 변경 : `os.chdir("가고 싶은 경로")`

In [13]:
# 경로에서 파일명만 확인
os.path.basename(filePath)

'bigkinds_NewsData.xlsx'

In [36]:
df = pd.read_excel("data/bigkinds_NewsData.xlsx")
df.head(2)

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,2.100201e+06,20220316,머니투데이,강상구|MTN PD,[갈 테마 가라] 2분기엔 턴어라운드 미리 선점해야 할 반도체 탑픽은?,경제>반도체,경제>유통,경제>자동차,NaN,NaN,NaN,NaN,신사,"머니투데이방송,MTNW,PMI,SK하이닉스","2분기,턴어라운드,선점,반도체,탑픽,진행,앵커,최희정,머니투데이방송,출연,어드바이저...","반도체,김진만,반도체주,최희정,머니투데이방송,구체화,턴어라운드,하이닉스,sic,성수...",[머니투데이 강상구 MTN PD] \n\n\n진행 : 최희정 머니투데이방송 앵커\n...,http://news.moneytoday.co.kr/view/mtview.php?n...,NaN
1,2.100601e+06,20220316,한국경제,강경주,"인텔, 작정하고 유럽에 100조 넘게 '반도체 투자'",경제>반도체,경제>자동차,경제>유통,NaN,NaN,NaN,"겔싱어,우르줄라","프랑스,미국,폴란드,고성,유럽,대유럽,바르셀로나엔,아시아,스페인,최고경영자,마그데부...","EU,인텔,라이엔,유럽연합(","인텔,작정,유럽,100조,반도체,투자,미국,반도체,기업,인텔,10년,유럽,반도체,생...","인텔,반도체,유로,eu,유럽,미국,폴란드,프랑스,우르줄라,파운드리,겔싱어,바르셀로나...",미국 반도체 기업 인텔이 앞으로 10년간 유럽에 반도체 생산과 연구 개발(R&D)을...,https://www.hankyung.com/economy/article/20220...,NaN


## 2. 데이터 확인

In [38]:
# 데이터 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2412 entries, 0 to 2411
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   뉴스 식별자             2412 non-null   float64
 1   일자                 2412 non-null   int64  
 2   언론사                2412 non-null   object 
 3   기고자                2367 non-null   object 
 4   제목                 2412 non-null   object 
 5   통합 분류1             2412 non-null   object 
 6   통합 분류2             2284 non-null   object 
 7   통합 분류3             1963 non-null   object 
 8   사건/사고 분류1          184 non-null    object 
 9   사건/사고 분류2          16 non-null     object 
 10  사건/사고 분류3          4 non-null      object 
 11  인물                 1453 non-null   object 
 12  위치                 2176 non-null   object 
 13  기관                 2403 non-null   object 
 14  키워드                2412 non-null   object 
 15  특성추출(가중치순 상위 50개)  2412 non-null   object 
 16  본문                 2412 

In [39]:
df.drop(columns=[
                "뉴스 식별자", "기고자", "통합 분류1", "통합 분류2", "통합 분류3", 
                "사건/사고 분류1", "사건/사고 분류2", "사건/사고 분류3", "인물", 
                "위치", "기관", "키워드", "특성추출(가중치순 상위 50개)", "본문",
                "URL", "분석제외 여부"], inplace=True)
df.head(2)

,일자,언론사,제목
0,20220316,머니투데이,[갈 테마 가라] 2분기엔 턴어라운드 미리 선점해야 할 반도체 탑픽은?
1,20220316,한국경제,"인텔, 작정하고 유럽에 100조 넘게 '반도체 투자'"


In [40]:
df.drop(columns=["언론사"], inplace=True)

In [42]:
# 일자, 제목 컬럼 확인
df.head(2)

,일자,제목
0,20220316,[갈 테마 가라] 2분기엔 턴어라운드 미리 선점해야 할 반도체 탑픽은?
1,20220316,"인텔, 작정하고 유럽에 100조 넘게 '반도체 투자'"


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2412 entries, 0 to 2411
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   일자      2412 non-null   int64 
 1   제목      2412 non-null   object
dtypes: int64(1), object(1)
memory usage: 37.8+ KB


In [45]:
# 결측치 확인
df.isnull().sum()

일자    0
제목    0
dtype: int64

## 3. train, test dataset 생성

In [48]:
# train, test set 분리
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df)

In [49]:
# train_data 확인
train_data.head(2)

,일자,제목
2164,20211224,"'반도체 설계' 와이팜, 테스나 인수 무산에 3%대 하락"
11,20220316,"장덕현 삼성전기 사장 ""반도체기판이 뉴 패러다임 신사업도 검토"""


In [52]:
# 데이터 개수 : 1809
# 컬럼 수 : 2
train_data.shape

(1809, 2)

In [53]:
# test_data 확인
test_data.head(2)

,일자,제목
490,20220223,"인천, 수출 반도체 의약품 철강판 일제히 호조 작년 상승세 이어가"
1987,20211230,"1,300만 주민 봉쇄 일주일 中 시안 삼성전자 반도체 공장도 차질"


In [54]:
# 데이터 개수 : 603
# 컬럼 수 : 2
test_data.shape

(603, 2)

## 4. 한자가 들어간 데이터 확인

In [55]:
from collections import Counter

In [58]:
len(train_data)

1809

In [73]:
train_hanja_data = train_data.copy()

In [78]:
pattern = re.compile('[一-龥]')

In [88]:
# 한자 리스트 확인
hanja_list = train_hanja_data["제목"].str.findall(pattern).sum()
hanja_list[:5]

['株', '中', '伊', '韓', '韓']

In [90]:
# 한자 중복값 제거
new_list = []
for i in hanja_list:
    if i not in new_list:
        new_list.append(i)
print(new_list)

['株', '中', '伊', '韓', '社', '對', '美', '日', '車', '獨', '文', '印', '價', '安', '外', '人', '亂', '産', '兆', '全', '企', '市', '國', '亞', '尹', '弗', '發', '史', '證', '夢', '論', '檢', '好', '說', '大', '年', '乙', '前', '稅', '李', '法', '新', '比', '硏', '戰', '號', '野']


In [59]:
k = []
for i in range(0, len(train_data)):
    a = re.findall('[一-龥]', train_data["제목"][i])
    if len(a) != 0:
        k = [*k, *a]

Counter(k).most_common()

KeyError: 1